In [2]:
import numpy as np
import pandas as pd
from model.autoencoder import AutoEncoder
from model.vae import VAE
import torch
import torch.nn.functional as F

In [14]:
pd_atac = pd.read_csv('./data/expr_atac_processed.csv', index_col=0)
pd_rna = pd.read_csv('./data/expr_rna_processed.csv', index_col=0)

In [18]:
atac = pd_atac.to_numpy()
rna = pd_rna.to_numpy()

In [19]:
atac.shape

(2641, 9467)

In [20]:
rna.shape

(2641, 1185)

In [21]:
def pairwise_distances(x):
    x_norm = (x**2).sum(1).view(-1, 1)
    y = x
    y_norm = x_norm.view(1, -1)

    dist = x_norm + y_norm - 2.0 * torch.mm(x, torch.transpose(y, 0, 1))
    return dist 

In [23]:
def loss(recon_atac, recon_rna, atac, rna, z, diff_distance, lamb):
    loss = F.mse_loss(rna, recon_rna) + F.mse_loss(atac, recon_atac) + lamb * F.mse_loss(diff_distance.reshape(-1), pairwise_distances(x).reshape(-1))
    return loss